Project 1

In [135]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
#API use and web scrapping
import requests
from bs4 import BeautifulSoup

In [63]:
sqlite_path = 'data/raw_data_project_m1.db'
conn_str = f'sqlite:///{sqlite_path}'
engine = create_engine(conn_str)

career_info 

In [64]:
query = 'SELECT * FROM career_info;'

In [65]:
career_info = pd.read_sql_query(query, engine)
career_info.head()

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f


Normalized Job Titles

In [75]:
list_api = []
list_job_code = set(career_info['normalized_job_code'])

In [79]:
for i in list_job_code:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    result_jobs = response.json()
    list_api.append(result_jobs)
    
data_jobs = pd.DataFrame(list_api)
data_jobs.head()
    

,uuid,title,normalized_job_title,parent_uuid,error
0,25fe195cbf915c0824fa6c44e6d0008c,Maintenance Data Analyst,maintenance data analyst,9b44dbc6767274d8b3a820e133803d00,NaN
1,3def30a9bc4356164176ba909f40bab8,Data Input Clerk,data input clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN
2,31bf3181813e21ffdfd76b5881fab066,Data Entry Supervisor,data entry supervisor,3ffe9bc56a3fd00b02fbdc71c03976f9,NaN
3,2f704538f6c9d6459a97314be98124e2,Data Collector,data collector,b9395a0c9f252a057e9b153c5c7f4bdd,NaN
4,ac2588d5d7676a49bdcd038e4a0a0ec5,Data Compiler,data compiler,325e2251e20170928426473156bd3c48,NaN


country_info

In [80]:
query2 = 'SELECT * FROM country_info;'
country_info = pd.read_sql_query(query2, engine)
country_info.head()

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city


personal_info

In [82]:
query3 = 'SELECT * FROM personal_info;'
personal_info = pd.read_sql_query(query3, engine)

poll_info

In [84]:
query4 = 'SELECT * FROM poll_info;'
poll_info = pd.read_sql_query(query4, engine)

API

In [86]:
response = requests.get('http://api.dataatwork.org/v1/jobs')

json_data = response.json()

In [71]:
job_info = pd.DataFrame(json_data)

MERGED DF

In [48]:
df_concat = pd.concat([country_info, career_info, job_info], ignore_index=True)

In [49]:
df_concat.head()

,uuid,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,title,normalized_job_title,parent_uuid,links
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_concat.drop(['dem_education_level', 'dem_full_time_job','parent_uuid','links'],
  axis='columns', inplace=True)

In [51]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19319 entries, 0 to 19318
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   uuid                  19318 non-null  object
 1   country_code          9649 non-null   object
 2   rural                 9649 non-null   object
 3   normalized_job_code   5702 non-null   object
 4   title                 20 non-null     object
 5   normalized_job_title  20 non-null     object
dtypes: object(6)
memory usage: 905.7+ KB


In [53]:
df_concat.isnull().sum()

uuid                        1
country_code             9670
rural                    9670
normalized_job_code     13617
title                   19299
normalized_job_title    19299
dtype: int64

In [57]:
df = df_concat.dropna(axis=0, subset=['normalized_job_code'])
df.isnull().sum()

uuid                       0
country_code            5702
rural                   5702
normalized_job_code        0
title                   5702
normalized_job_title    5702
dtype: int64

Web sacrapping

In [91]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
html[0:600]   

b'\xef\xbb\xbf<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<meta http-equiv="X-UA-Compatible" content="Ie=Edge,chrome=1"/> <meta charset="UTF-8" /><title>Glossary:Country codes - Statistics Explained</title>\n<meta name="generator" content="MediaWiki 1.22.5" />\n<link rel="shortcut icon" href="https://ec.europa.eu/eurostat/estat-theme/images/favicon.ico" />\n<link rel="search" type="application/opensearchdescription+xml" href="/eurostat/statistics-explained/opensearch_desc.php" title="Statistics Explained (en)" />\n<link rel="EditURI" type="application/rsd+xml" href="https://ec.europ'

In [130]:
soup = BeautifulSoup(html, 'html')

In [132]:
country_codes = pd.read_html(url)

In [139]:
country_codes['0']

TypeError: list indices must be integers or slices, not str

In [137]:
df_country_codes = np.array([v for v in country_codes])

<ipython-input-137-fc5870e2e382>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  df_country_codes = np.array([v for v in country_codes])
